In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, GlobalAveragePooling2D
import os
import cv2
import pickle
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import Model


from imblearn.under_sampling import RandomUnderSampler
from keras.utils.np_utils import to_categorical
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import time

%load_ext tensorboard

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
dataPath = "../../data/OCT/OCT2017/"

In [4]:
labels_available = os.listdir(os.path.join(dataPath, "train"))
print("Total Number of Classes Detected :",len(labels_available))

labels_list = ['NORMAL',"CNV","DME","DRUSEN"]

Total Number of Classes Detected : 4


In [5]:
images=[]
y_trn=[]
for x in labels_list:
    xPath = os.path.join(dataPath, "train", x)
    myPicList = os.listdir(xPath)
    for y in myPicList:
        images.append(cv2.imread(os.path.join(xPath, y)))
        y_trn.append(labels_list.index(x))
    print(x ,end=" ")

y_train = to_categorical(y_trn,len(labels_list))

NORMAL CNV DME DRUSEN 

In [6]:
x_val=[]
y_val=[]
for x in labels_list:
    xPath = os.path.join(dataPath, "val", x)
    myPicList = os.listdir(xPath)
    for y in myPicList:
        x_val.append(cv2.imread(os.path.join(xPath, y)))
        y_val.append(labels_list.index(x))
    print(x ,end=" ")

y_validation = to_categorical(y_val,len(labels_list))

NORMAL CNV DME DRUSEN 

In [7]:
x_tst=[]
y_tst=[]
for x in labels_list:
    xPath = os.path.join(dataPath, "test", x)
    myPicList = os.listdir(xPath)
    for y in myPicList:
        x_tst.append(cv2.imread(os.path.join(xPath, y)))
        y_tst.append(labels_list.index(x))
    print(x ,end=" ")

NORMAL CNV DME DRUSEN 

In [8]:
def resizeIm(im, size):
    if im.shape[2] == 1:
        im = cv2.cvtColor(im,cv2.COLOR_GRAY2RGB)
    return cv2.resize(im, size)

def resizeIms(x, size):
    return np.array(list(map(lambda im: resizeIm(im, size), x)))

# Substract the mean specified by Mean (https://www.mathworks.com/help/deeplearning/ref/nnet.cnn.layer.imageinputlayer.html)
# Substract mean by channel
def zerocenter(x):
    return (x - np.mean(x, axis=(0, 1))) / (2*np.std(x, axis=(0, 1)))

# Rescale the input to be in the range [-1, 1] using the minimum and maximum values specified by Min and Max, respectively. (https://www.mathworks.com/help/deeplearning/ref/nnet.cnn.layer.imageinputlayer.html)
# Compute using the formula: $x\prime = 2 * \frac{x - \min{x}}{\max{x} - \min{x}} - 1$ taken from: https://stats.stackexchange.com/a/178629
def rescaleSymmetric(x):
    x = zerocenter(x)
    return 2 * (x - np.min(x, axis=(0, 1)))/np.ptp(x, axis=(0, 1)) - 1

def normalizeIms(x, normalization):
    if normalization == "zerocenter":
        func = zerocenter
    elif normalization == "rescale-symmetric":
        func = rescaleSymmetric
    else:
        return x
    return np.array(list(map(lambda im: func(im), x)))

In [9]:
class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self,fraction, model):
        super(CustomCallback,self).__init__()
        self.fraction = fraction
        self.train_a = [];
        self.val_a =[];
        self.logPath = os.path.join(model, "log.txt")

        if not os.path.isdir(model):
            os.mkdir(model)

        with open(self.logPath,'w') as f:
            f.write('Starting of logging..\n')

        self.fig = plt.figure(figsize=(4,3))
        self.ax = plt.subplot(1,1,1)
        plt.ion()

    def on_train_begin(self,logs=None):
        self.fig.show()
        self.fig.canvas.draw()

    def on_train_end(self,logs=None):
        with open(self.logPath,'a') as f:
              f.write('End of logging..\n')
    
    def on_epoch_begin(self,epoch,logs=None):
        lr= tf.keras.backend.get_value(self.model.optimizer.lr)
        lr *= self.fraction
        tf.keras.backend.set_value(self.model.optimizer.lr,lr)
        with open(self.logPath,'a') as f:
            f.write('At epoch {:02d}, learning rate changed to {:.4f}\n'.format(epoch,lr))
    
    def on_epoch_end(self,epoch,logs=None):
        val_acc = logs.get('val_accuracy')
        train_acc = logs.get('accuracy')
        self.train_a.append(train_acc)
        self.val_a.append(val_acc)
        with open(self.logPath,'a') as f:
            f.write('At epoch {:02d}, training accuracy: {:.3f}, validation accuracy: {:.3f}\n'.format(epoch,train_acc,val_acc))
        self.ax.clear()
        self.ax.plot(range(1+epoch),self.train_a,label="Training")
        self.ax.plot(range(1+epoch),self.val_a,label="Validation")
        self.ax.set_xlabel('Epochs')
        self.ax.set_ylabel('Accuracy')
        self.ax.legend()
        self.fig.canvas.draw()
        self.fig.show()
        
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

## Load models

In [10]:
# xception base model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input, decode_predictions
xceptionNetModel = Xception(weights='imagenet')

# opticnet base model
opticNetModel = tf.keras.models.load_model('../Optic_net-4_classes-Kermany2018.hdf5')

# resnet base model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

resNetModel = ResNet50(weights='imagenet')

In [11]:
def emptyModelGenerator(model, newWeights=False, lastOnly=False):
    model = model.lower()
    if model == "resnet":
        x = resNetModel.output
        predictions = Dense(len(labels_list), activation='softmax')(x)
        newModel = Model(inputs=resNetModel.input, outputs=predictions)
        size = (224, 224)
        normalization = "zerocenter"
    elif model == "xception":
        x = xceptionNetModel.output
        predictions = Dense(len(labels_list), activation='softmax')(x)
        newModel = Model(inputs=xceptionNetModel.input, outputs=predictions)
        size = (299, 299)
        normalization = "rescale-symmetric"
    elif model == "opticnet":
        newModel = tf.keras.models.load_model('../Optic_net-4_classes-Kermany2018.hdf5')
        size = (224, 224)
        normalization = "rescale-symmetric"
    if newWeights:
        newModel = tf.keras.models.clone_model(newModel)
    if lastOnly:
        newModel = lastDenseTrainable(newModel) # block all layer except the last dense ones
    return newModel, size, normalization


def lastDenseTrainable(model):
    model.trainable = False
    for i in range(1, len(model.layers)):
        if model.layers[-i].__class__.__name__ == "Dense":
            model.layers[-i].trainable= True
        elif model.layers[-i].__class__.__name__ == "Activation":
            continue
        else:
            break
    return model
    

def testPredict(model, size, name=None, normalization=None):
    X_test = resizeIms(x_tst, size)
    if normalization:
        X_test = normalizeIms(X_test, normalization)
    X_test = np.array(X_test)
    Y_test = np.array(y_tst)
    
    prediction = model.predict(X_test)
    preds = np.argmax(prediction, axis=1)
    acc = sum(preds == Y_test) / len(Y_test)
    print(f'Test acc for {name if name else "model"}: {acc:.6f}')
    return acc
    

def computeConfussionMatrix(predictions, labels):
    num_labels = len(labels_list)
    cMatrix = np.zeros(shape=(num_labels, num_labels))
    for i in len(predictions):
        p = int(predictions[i])
        t = int(predictions[i])
        cMatrix[t, p] += 1
    print(cMatrix)
    ax = sns.heatmap(cMatrix, cmap="Blues", annot=True, fmt="d", xticklabels=labels_list, yticklabels=labels_list)
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.title("Confusion matrix")
    plt.show();
    return cMatrix
    

In [12]:
models = ["xception", "resnet", "opticnet"]

for mod in models:
    print("\n\n" + ("*"*50) + "\nModel: " + mod + '\n' + ("*"*50))
    m = emptyModelGenerator(mod, newWeights=False, lastOnly=True)[0]
    #m.trainable = False
    a = 0
    for l in m.layers:
        #l.trainable = True
        if l.trainable:
            print(l.name, l.trainable)
            a += 1
            #print(l.trainable_weights)
    print(f"\nTotal: {len(m.layers)} trainable: {a}")
    #print(m.summary())



**************************************************
Model: xception
**************************************************
predictions True
dense True

Total: 135 trainable: 2


**************************************************
Model: resnet
**************************************************
predictions True
dense_1 True

Total: 178 trainable: 2


**************************************************
Model: opticnet
**************************************************
Dense_1 True
Dense_2 True

Total: 276 trainable: 2


In [13]:
lr = 0.001
momentum = 0.5
optim = Adam(learning_rate=lr)
#optim = SGD(learning_rate=lr, momentum=momentum)
epochs = 30
batch_size = 50

In [14]:
import pandas as pd
results = pd.read_csv("Results.csv")
results["normalization"] = None

In [ ]:
from sklearn.model_selection import train_test_split
import datetime

maxTrain = len(y_train)
#for newWeights in [False, True]:
for newWeigths in False:
    # if is pretrained, train both cases, otherwise train all
    #tLayer = [False] if newWeights else [True, False]
    for trainLastLayerOnly in tLayer:
        for p in [0.01, 0.025, 0.05, 0.075, 0.09, 0.1, 0.25, 0.4, 0.5, 0.6, 0.75, 0.9, 1.0]:
            #X_trn, X_tst, y_trn, y_tst
            if p < 1:
                X_t, _, y_t, _ = train_test_split(images, y_train, test_size=1-p, random_state=234)
            else:
                X_t = images; y_t = y_train;
            print(f"Labels fraction: {sum(y_t == 1) / sum(y_train == 1)}")
            for net in ["xception", "resnet", "opticnet"]:
                print(f"Training {net} for {p}% of train size (aka {len(X_t)} images) with pretrained: {not newWeights} and onlyLastLayersTrained: {trainLastLayerOnly}...")
                model, size, normalization = emptyModelGenerator(net, newWeights=newWeights, lastOnly=trainLastLayerOnly)
                X_trn = resizeIms(X_t, size)
                #X_trn = normalizeIms(X_trn, normalization=normalization)
                X_val = resizeIms(x_val, size)
                #X_val = normalizeIms(X_val, normalization=normalization)
                log_dir = f"logs/{net}/fit/{p}trainSet_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
                tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
                optim = Adam(learning_rate=0.001)
                time_callback = TimeHistory()
                model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['accuracy'])
                hist = model.fit(X_trn, y_t, epochs=epochs, validation_data = (X_val, y_validation), batch_size=batch_size,
                            shuffle=True, max_queue_size=20,
                            use_multiprocessing=True, workers=5, 
                            callbacks=[CustomCallback(fraction=0.9, model=net), tensorboard_callback, time_callback])
                trainTime = sum(time_callback.times)
                model.save(f"../{net}/{net}_{epochs}epochs_{p*maxTrain}_images_{newWeights}_newWeights_{trainLastLayerOnly}_lastLayerOnly_{normalization}_normalization")
                tstAcc = testPredict(model, size, name=net, normalization=normalization)
                results = results.append({
                    'model': net, 
                    'train set images': len(X_t), 
                    'pretrained': not newWeights, 
                    'pretrained dataset': None if newWeights else ("OCT2017,Srinivasan2014" if net == "opticnet" else "Imagenet"),
                    'Trained layers': "Last dense" if trainLastLayerOnly else "All", 
                    'epochs': epochs, 
                    'batch size': batch_size, 
                    'learning rate': lr, 
                    'optimizer': optim._name,
                    'training time (seconds)': trainTime, 
                    'train accuracy': hist.history["accuracy"][-1], 
                    'train loss': hist.history["loss"][-1],
                    'validation accuracy': hist.history["val_accuracy"][-1], 
                    'validation loss': hist.history["val_loss"][-1], 
                    'test accuracy': tstAcc,
                    #'normalization': normalization
                }, ignore_index=True)
                results.to_csv("Results_temp.csv", index=False)
                del model
                del X_trn
                del X_val
                print("Done!\n" + '-'*50, end='\n\n')

Labels fraction: [0.0101463  0.00983739 0.01039831 0.00963324]
Training xception for 0.01% of train size (aka 834 images) with pretrained: True and onlyLastLayersTrained: False...


<ipython-input-9-a21c27d31b2a>:20: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  self.fig.show()


Epoch 1/30
17/17 [==============================] - 22s 345ms/step - loss: 1.3677 - accuracy: 0.4723 - val_loss: 1.3847 - val_accuracy: 0.2500


<ipython-input-9-a21c27d31b2a>:48: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  self.fig.show()


Epoch 2/30
17/17 [==============================] - 2s 122ms/step - loss: 1.3556 - accuracy: 0.4667 - val_loss: 1.3793 - val_accuracy: 0.3438
Epoch 3/30
17/17 [==============================] - 2s 122ms/step - loss: 1.3404 - accuracy: 0.4876 - val_loss: 1.3739 - val_accuracy: 0.4062
Epoch 4/30
17/17 [==============================] - 2s 126ms/step - loss: 1.3227 - accuracy: 0.5417 - val_loss: 1.3666 - val_accuracy: 0.4375
Epoch 5/30
17/17 [==============================] - 2s 123ms/step - loss: 1.3131 - accuracy: 0.5571 - val_loss: 1.3626 - val_accuracy: 0.4375
Epoch 6/30
17/17 [==============================] - 2s 122ms/step - loss: 1.2986 - accuracy: 0.5705 - val_loss: 1.3618 - val_accuracy: 0.4062
Epoch 7/30
17/17 [==============================] - 2s 123ms/step - loss: 1.2885 - accuracy: 0.5823 - val_loss: 1.3601 - val_accuracy: 0.4375
Epoch 8/30
17/17 [==============================] - 2s 123ms/step - loss: 1.2887 - accuracy: 0.5668 - val_loss: 1.3590 - val_accuracy: 0.4062
Epoch 

...

Epoch 2/30
1253/1253 [==============================] - 95s 76ms/step - loss: 1.2300 - accuracy: 0.4466 - val_loss: 1.5563 - val_accuracy: 0.2500
Epoch 3/30
1253/1253 [==============================] - 95s 76ms/step - loss: 1.2355 - accuracy: 0.4412 - val_loss: 1.5658 - val_accuracy: 0.2500
Epoch 4/30
1253/1253 [==============================] - 96s 77ms/step - loss: 1.2335 - accuracy: 0.4436 - val_loss: 1.5582 - val_accuracy: 0.2500
Epoch 5/30
1253/1253 [==============================] - 96s 77ms/step - loss: 1.2286 - accuracy: 0.4473 - val_loss: 1.5504 - val_accuracy: 0.2500
Epoch 6/30
1253/1253 [==============================] - 95s 76ms/step - loss: 1.2339 - accuracy: 0.4445 - val_loss: 1.5619 - val_accuracy: 0.2500
Epoch 7/30
1253/1253 [==============================] - 96s 76ms/step - loss: 1.2288 - accuracy: 0.4483 - val_loss: 1.5543 - val_accuracy: 0.2500
Epoch 8/30
1253/1253 [==============================] - 95s 76ms/step - loss: 1.2302 - accuracy: 0.4493 - val_loss: 1.5566 -

<ipython-input-9-a21c27d31b2a>:20: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  self.fig.show()


Epoch 1/30
1253/1253 [==============================] - 1518s 1s/step - loss: 1.8208 - accuracy: 0.6687 - val_loss: 0.3072 - val_accuracy: 0.8750


<ipython-input-9-a21c27d31b2a>:48: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  self.fig.show()


Epoch 2/30
1253/1253 [==============================] - 1497s 1s/step - loss: 0.2739 - accuracy: 0.9097 - val_loss: 0.3553 - val_accuracy: 0.8438
Epoch 3/30
1253/1253 [==============================] - 1498s 1s/step - loss: 0.2021 - accuracy: 0.9337 - val_loss: 0.6162 - val_accuracy: 0.6562
Epoch 4/30
1253/1253 [==============================] - 1496s 1s/step - loss: 0.1734 - accuracy: 0.9411 - val_loss: 0.2797 - val_accuracy: 0.9062
Epoch 5/30
1253/1253 [==============================] - 1493s 1s/step - loss: 0.1542 - accuracy: 0.9476 - val_loss: 0.0237 - val_accuracy: 1.0000
Epoch 6/30
1253/1253 [==============================] - 1498s 1s/step - loss: 0.1385 - accuracy: 0.9519 - val_loss: 0.0373 - val_accuracy: 1.0000
Epoch 7/30
1253/1253 [==============================] - 1492s 1s/step - loss: 0.1188 - accuracy: 0.9587 - val_loss: 0.0613 - val_accuracy: 1.0000
Epoch 8/30
1253/1253 [==============================] - 1493s 1s/step - loss: 0.1017 - accuracy: 0.9647 - val_loss: 0.0491 -